
# Notebook for U-Net and Attention U-Net models

___

## PAPER: Comparison of Conventional Machine Learning and Convolutional Deep Learning models for Seagrass Mapping using Satellite Imagery

#### Antonio Mederos-Barrera (mederosbarrera.antonio@gmail.com)

### Imports

In [ ]:
# General
import os
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import random
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
import scipy.io
from scipy.stats import mode
from torch.optim.lr_scheduler import ExponentialLR

# Utils
sys.path.append(os.path.abspath('.'))
from utils.class_weights import obtain_class_weights
from utils.dice_loss_function import dice_loss
from utils.early_stopping import EarlyStopping
from utils.error_metrics import error_metrics
from utils.load_data import find_files, mean_std_dataset, data_train_test, SegmentationDataset
from utils.map_estimation_functions import mode_no_zeros, numpy_to_torch, obtain_mode_no_zeros, predict_model, sliding_window
from utils.model_initialization import initialize_weights_kaiming
from models.unet import UNet, DoubleConv, Down, Up, OutConv
from models.patchgan import PatchGAN

### Hyperparameters

In [ ]:
# GPU use
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)
PIN_MEMORY = True if DEVICE == "cuda" else False

# Reproducibility
def set_manual_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
SEED = 0                                                  # Random seed for reproducibility
set_manual_seed(SEED)

BATCH_SIZE = "all"                                        # Batch size (can be "all" to use all the training and test dataset)

NUM_EPOCHS = 2000                                         # Epochs (upper limit in case Early stopping does not converge)

PATIENCE_EARLYSTOPPING = 140                              # Patiente of Early Stopping

INPUT_WIDTH = 70                                          # Image size (first dimension)
INPUT_HEIGHT = 70                                         # Image size (second dimension)
NUM_CHANNELS = 8                                          # Image channels (third dimension)

# Path to the dataset
IMAGE_DATASET_PATH = "C:/Path/To/Data"
NUM_CLASSES = 5                                           # Number of classes
VECTOR_IDX_TEST = [6,10,9,7,13,17,19,25,29,34,37,42,46,50]# Vector images for the test dataset

PATH = "C:/Path/To/Save/Results/"
MODEL_PATH = PATH + "unet_"
PLOT_PATH = PATH + "plot_"
NPY_PATH = PATH + "lossfunc_"

# Models hyperparameters (LR, L2, ExpSchedGamma, AttGates, LambdaL1)
Model1 = [0.001, 0.0001, None, False, 100]

Models = [Model1]

# Estimation
IMG_PATH = 'C:Path/To/Cmomplete/WorldView/Image.mat' # Image
mymap = np.array([  # Colors
    [0, 0, 0],      # Land
    [139, 0, 0],    # Red algae
    [0, 139, 1],    # C. nodosa (sebadal)
    [142, 87, 2],   # Rock
    [255, 255, 0]   # Sand
])
window_size = (70, 70) # Slide windows
stride = 5          # Slide windows

 ### Training and test

In [ ]:

def split_params_for_weight_decay(model):
    decay, no_decay = [], []
    for n, p in model.named_parameters():
        if not p.requires_grad:
            continue
        is_norm = any(k in n.lower() for k in ["bn", "norm", "bias"])
        is_att  = "att" in n.lower()
        if is_norm or is_att:
            no_decay.append(p)
        else:
            decay.append(p)
    return decay, no_decay

def plot_error_curve(matrix_data_train_test, limit_G=None, limit_L1=None):
    md = matrix_data_train_test.copy()
    aux_G = md[:, 0]
    aux_L1 = md[:, 1]
    if limit_G is not None:
        aux_G[aux_G > limit_G] = limit_G
    if limit_L1 is not None:
        aux_L1[aux_L1 > limit_L1] = limit_L1
    plt.style.use("ggplot")
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    axes[0].plot(aux_G, label="Training loss (G_total)", color="tab:blue")
    axes[0].set_xlabel("Epoch #")
    axes[0].set_ylabel("Loss")
    axes[0].set_title("Training Loss (G_total)")
    axes[0].legend(loc="upper right")
    axes[1].plot(aux_L1, label="Val L1", color="tab:orange")
    axes[1].set_xlabel("Epoch #")
    axes[1].set_ylabel("Loss")
    axes[1].set_title("Validation L1 Loss")
    axes[1].legend(loc="upper right")
    plt.tight_layout()
    plt.show()

def delete_file(directory, start, end):
    for file in os.listdir(directory):
        if file.startswith(start) and file.endswith(end):
            file_path = os.path.join(directory, file)
            if os.path.isfile(file_path):
                try:
                    os.remove(file_path)
                except OSError as e:
                    print(f"Error deleting the file {file_path}: {e}")

trainLoader, testLoader, trainDS, testDS, BATCH_SIZE_TRAIN, BATCH_SIZE_TEST = data_train_test(IMAGE_DATASET_PATH,
                                                                                              INPUT_WIDTH,
                                                                                              INPUT_HEIGHT,
                                                                                              NUM_CLASSES,
                                                                                              BATCH_SIZE,
                                                                                              PIN_MEMORY,
                                                                                              VECTOR_IDX_TEST)
class_weights = obtain_class_weights(trainLoader, NUM_CLASSES, DEVICE)


indx_print = 0

criterion_GAN = nn.BCEWithLogitsLoss()
criterion_L1 = nn.L1Loss()

for mdl_hyp in Models:
    
    lr_value = mdl_hyp[0]
    reg_L2_value = mdl_hyp[1]
    scheduler_gamma = mdl_hyp[2]
    use_attention_gates = mdl_hyp[3]
    lambda_L1 = mdl_hyp[4]
    
    # Initial print
    indx_print += 1
    print("[",indx_print,"of",len(Models),"]")

    # Random seed
    set_manual_seed(SEED)

    # Model
    G = UNet(n_channels=NUM_CHANNELS, n_classes=NUM_CLASSES, use_attention_gates=use_attention_gates)
    D = PatchGAN(in_channels=NUM_CHANNELS + NUM_CLASSES)
    
    initialize_weights_kaiming(G)
    G.to(DEVICE)
    initialize_weights_kaiming(D)
    D.to(DEVICE)

    # Optimizer
    if reg_L2_value is None:
        opt_G = Adam(G.parameters(), lr=lr_value)
        opt_D = Adam(D.parameters(), lr=lr_value/2)
    else:
        opt_G = Adam(G.parameters(), lr=lr_value,   weight_decay=reg_L2_value)
        opt_D = Adam(D.parameters(), lr=lr_value/2)

    # Scheduler LR
    if scheduler_gamma is not None:
        scheduler_G = ExponentialLR(opt_G, gamma=scheduler_gamma)
        scheduler_D = ExponentialLR(opt_D, gamma=scheduler_gamma)
        lrs_values_G, lrs_values_D = [], []

    # Data
    trainLoader, testLoader, trainDS, testDS, BATCH_SIZE_TRAIN, BATCH_SIZE_TEST = data_train_test(IMAGE_DATASET_PATH,
                                                                                                  INPUT_WIDTH,
                                                                                                  INPUT_HEIGHT,
                                                                                                  NUM_CLASSES,
                                                                                                  BATCH_SIZE,
                                                                                                  PIN_MEMORY,
                                                                                                  VECTOR_IDX_TEST)

    # Steps
    trainSteps = len(trainDS) // BATCH_SIZE_TRAIN
    testSteps  = len(testDS)  // BATCH_SIZE_TEST

    # Early Stopping
    early_stopping = EarlyStopping(patience=PATIENCE_EARLYSTOPPING,
                                   path_base=MODEL_PATH,
                                   mode='min',
                                   loss_func="pix2pix",
                                   lr_value=lr_value,
                                   scheduler_gamma=scheduler_gamma,
                                   verbose=False,
                                   min_delta=0.00001,
                                   reg_L2_value=reg_L2_value,
                                   use_attention_gates=use_attention_gates)

    # Training and test epochs
    matrix_data_train_test = np.zeros((NUM_EPOCHS, 2))
    for e in tqdm(range(NUM_EPOCHS)):

        # Train
        G.train()
        D.train()
        
        total_G_loss = 0.0
        total_val_L1 = 0.0
        for (i, (x, y)) in enumerate(trainLoader):
            (x, y) = (x.to(DEVICE), y.to(DEVICE))

            # Discriminator Training
            opt_D.zero_grad()
            with torch.no_grad():
                fake_y = G(x)
            pred_real = D(x, y)
            pred_fake = D(x, fake_y.detach())
            loss_D_real = criterion_GAN(pred_real, torch.ones_like(pred_real))
            loss_D_fake = criterion_GAN(pred_fake, torch.zeros_like(pred_fake))
            loss_D = 0.5 * (loss_D_real + loss_D_fake)
            loss_D.backward()
            opt_D.step()

            # Generator Training
            opt_G.zero_grad()
            fake_y = G(x)
            pred_fake_for_G = D(x, fake_y)
            loss_G_GAN = criterion_GAN(pred_fake_for_G, torch.ones_like(pred_fake_for_G))
            loss_G_L1  = criterion_L1(fake_y, y) * lambda_L1
            loss_G = loss_G_GAN + loss_G_L1
            loss_G.backward()
            opt_G.step()

            total_G_loss += loss_G.detach().item()

        # Test
        with torch.no_grad():
            G.eval()
            D.eval()
            for (i, (x, y)) in enumerate(testLoader):
                (x, y) = (x.to(DEVICE), y.to(DEVICE))
                fake_y = G(x)
                val_L1 = criterion_L1(fake_y, y)
                total_val_L1 += val_L1.item()
        
        # Training and test losses
        avg_G_loss = total_G_loss / trainSteps
        avg_val_L1 = total_val_L1 / testSteps
        matrix_data_train_test[e, 0] = avg_G_loss
        matrix_data_train_test[e, 1] = avg_val_L1

        # Actualization of Scheduler LR
        if scheduler_gamma is not None:
            lrs_values_G.append(scheduler_G.get_last_lr()[0])
            lrs_values_D.append(scheduler_D.get_last_lr()[0])
            scheduler_G.step()
            scheduler_D.step()

        # Early Stopping
        early_stopping.epoch = e
        early_stopping.lr = lr_value
        early_stopping(avg_val_L1, G)
        if early_stopping.early_stop:

            best_epoch = int(e-PATIENCE_EARLYSTOPPING)
            print("Early stopping in the epoch:", best_epoch)

            # Save
            NPY_PATH_ = NPY_PATH+'pix2pix_LR-'+str(lr_value)+'_ExpSchedGamma-'+str(scheduler_gamma)+'_L2-'+str(reg_L2_value)+'_EPOCHS-'+str(best_epoch)+"_EarlyStopping_AG-"+str(use_attention_gates)+".npy"
            np.save(NPY_PATH_, matrix_data_train_test[:best_epoch+1,:])
            NPY_PATH_ALL_ = NPY_PATH+'pix2pix_LR-'+str(lr_value)+'_ExpSchedGamma-'+str(scheduler_gamma)+'_L2-'+str(reg_L2_value)+'_EPOCHS-'+str(best_epoch)+"_EarlyStopping_AG-"+str(use_attention_gates)+"_ALL.npy"
            np.save(NPY_PATH_ALL_, matrix_data_train_test)

            # Error Curves
            MODEL_PATH_ = MODEL_PATH+'pix2pix_LR-'+str(lr_value)+'_ExpSchedGamma-'+str(scheduler_gamma)+'_L2-'+str(reg_L2_value)+'_EPOCHS-'+str(best_epoch)+"_EarlyStopping_AG-"+str(use_attention_gates)+".pth"
            print(MODEL_PATH_)
            plot_error_curve(matrix_data_train_test[:best_epoch+1,:])
            plot_error_curve(matrix_data_train_test[:best_epoch+1,:], limit_G=50.0, limit_L1=1.0)

            if scheduler_gamma is not None:
                # LRs values (Schedulers)
                lrs_values_G_np = np.array(lrs_values_G)
                lrs_values_D_np = np.array(lrs_values_D)
                plt.figure()
                plt.style.use("ggplot")
                plt.plot(lrs_values_G_np[:best_epoch+1], label="LR_G")
                plt.plot(lrs_values_D_np[:best_epoch+1], label="LR_D")
                plt.xlabel('Epochs #')
                plt.ylabel('Learning Rate')
                plt.title("LR Exponential Scheduler. Gamma: "+str(scheduler_gamma))
                plt.grid(True)
                plt.legend()
                plt.show()

            break

    # Early stopping is not triggered
    if not early_stopping.early_stop:
        print("Early stopping is not triggered")
        plot_error_curve(matrix_data_train_test)
        plot_error_curve(matrix_data_train_test, limit_G=50.0, limit_L1=1.0)

        if scheduler_gamma is not None:
            plt.figure()
            plt.style.use("ggplot")
            plt.plot(lrs_values_G, label="LR_G")
            plt.plot(lrs_values_D, label="LR_D")
            plt.xlabel('Epochs #')
            plt.ylabel('Learning Rate')
            plt.title("LR Exponential Scheduler. Gamma: "+str(scheduler_gamma))
            plt.grid(True)
            plt.legend()
            plt.show()

        MODEL_PATH_G_LAST_ = MODEL_PATH+'pix2pix_LR-'+str(lr_value)+'_ExpSchedGamma-'+str(scheduler_gamma)+'_L2-'+str(reg_L2_value)+'_EPOCHS-'+str(best_epoch)+"_EarlyStopping_AG-"+str(use_attention_gates)+".pth"
        torch.save(G.state_dict(), MODEL_PATH_G_LAST_)

### Error metrics

In [ ]:

FINAL_MODELS = [PATH+archivo for archivo in os.listdir(PATH) if archivo.startswith(MODEL_PATH[len(PATH):]) and archivo.endswith(".pth")]

CSV_PATH = [model.replace('unet_', 'csv_') for model in FINAL_MODELS]
CSV_PATH = [model.replace('.pth', '.csv') for model in CSV_PATH]

for indx_model, model in enumerate(FINAL_MODELS):

    print("[",indx_model+1,"of",len(FINAL_MODELS),"]")
    
    # Model
    mdl = torch.load(model, weights_only=False)
    mdl.to(DEVICE)
    print(model)
          
    # Metrics initialization
    accuracy = np.zeros((NUM_CLASSES,))
    precision = np.zeros((NUM_CLASSES,))
    recall = np.zeros((NUM_CLASSES,))
    f1 = np.zeros((NUM_CLASSES,))
    iou = np.zeros((NUM_CLASSES,))
    dice = np.zeros((NUM_CLASSES,))
                
    # Test error metrics
    with torch.no_grad():
        mdl.eval()
        testSteps = 0
        for (i, (x, y)) in enumerate(testLoader):
            (x, y) = (x.to(DEVICE), y.to(DEVICE))
            pred = mdl(x)
            pred = torch.argmax(pred, dim=1)
            pred = F.one_hot(pred, num_classes=x.shape[1])
            pred = pred.permute(0, 3, 1, 2)
            y = y.cpu().detach().numpy()
            pred = pred.cpu().detach().numpy()
                                
            accuracy_, precision_, recall_, f1_, iou_, dice_ = error_metrics(y, pred)
            accuracy += accuracy_
            precision += precision_
            recall += recall_
            f1 += f1_
            iou += iou_
            dice += dice_
            
            testSteps +=1
                    
        accuracy /= testSteps
        precision /= testSteps
        recall /= testSteps
        f1 /= testSteps
        iou /= testSteps
        dice /= testSteps
        
        # Values and mean without land class (not considered in ML models)
        accuracy =  np.concatenate((accuracy, [-1 if -1 in accuracy else np.mean(accuracy[1:])]))
        precision =  np.concatenate((precision, [-1 if -1 in precision else np.mean(precision[1:])]))
        recall =  np.concatenate((recall, [-1 if -1 in recall else np.mean(recall[1:])]))
        f1 =  np.concatenate((f1, [-1 if -1 in f1 else np.mean(f1[1:])]))
        iou =  np.concatenate((iou, [-1 if -1 in iou else np.mean(iou[1:])]))
        dice =  np.concatenate((dice, [-1 if -1 in dice else np.mean(dice[1:])]))
        print("Metrics (Acc, Prec, Recall, F1, IoU, Dice):\n", 
                accuracy,'\n',
                precision,'\n',
                recall,'\n',
                f1,'\n',
                iou,'\n',
                dice,'\n')

        # Save metrics in CSV file
        metrics_dict = {
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1-score': f1,
            'IoU': iou,
            'Dice Coefficient': dice
        }
        columns = ['Land', 'Red algae', 'C. nodosa', 'Rock', 'Sand', 'Mean (without land)']
        df_metrics = pd.DataFrame(metrics_dict, index=columns).transpose()
        df_metrics.to_csv(CSV_PATH[indx_model], sep=';', index=True)


### Map estimation

In [ ]:

FINAL_MODELS = [PATH+archivo for archivo in os.listdir(PATH) if archivo.startswith(MODEL_PATH[len(PATH):]) and archivo.endswith(".pth")]

SAVE_PATH = [model.replace('unet_', 'plot_') for model in FINAL_MODELS]
SAVE_PATH = [model.replace('.pth', '.mat') for model in SAVE_PATH]

mat_data_image = scipy.io.loadmat(IMG_PATH)
filtered_keys_image = [key for key in mat_data_image.keys() if "__" not in key]
image = mat_data_image[filtered_keys_image[0]]

mean, std = mean_std_dataset(IMAGE_DATASET_PATH, INPUT_WIDTH, INPUT_HEIGHT, NUM_CLASSES, BATCH_SIZE, PIN_MEMORY)
image = numpy_to_torch(image, mean, std)

# Estimation
for indx_model, path_model in tqdm(enumerate(FINAL_MODELS)):
    
    print("[",indx_model+1,"of",len(FINAL_MODELS),"]")
    
    # Model
    mdl = torch.load(path_model, weights_only=False)
    mdl.to(DEVICE)

    # Slide window
    pred_image = sliding_window(image, window_size, stride, mdl, DEVICE)

    # Mode without zeros values
    pred_image_mode = obtain_mode_no_zeros(pred_image)
    
    # Presentation
    plt.figure(figsize=(15, 15))
    plt.title(SAVE_PATH[indx_model])
    plt.imshow(mymap[pred_image_mode.astype(int)])
    plt.show()
    
    # Save
    scipy.io.savemat(SAVE_PATH[indx_model], {'seabedmap_graciosa':pred_image_mode})